In [4]:
import numpy as np

coords = np.array([[-1, -1], [1, -1], [1, 1], [-1, 1]])

center = np.mean(coords, axis=0)

distances = np.linalg.norm(coords - center, axis=1)

normalizedCoords = coords / distances[:, np.newaxis]

print(normalizedCoords)


[[-0.70710678 -0.70710678]
 [ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]
 [-0.70710678  0.70710678]]
